In [30]:
from tqdm import tqdm
import time
import json
from enum import Enum
from gurobipy import Model, GRB, quicksum, max_
import pandas as pd
from random import randint
import plotly.graph_objs as go
import numpy as np
from numpy import array


In [31]:
class DATASET(Enum):
    TOY = "toy"
    MEDIUM = "medium"
    LARGE = "large"

# Fonctions pour préparer le modèle


In [32]:
def load_data(name):
    """name must be an instance of DATASET like DATASET.TOY for example"""
    if not isinstance(name, DATASET):
        raise TypeError("direction must be an instance of DATASET Enum")
    with open(f"../data/{name.value}_instance.json", "r") as f:
        data = json.load(f)
    return data


def get_dims(data):
    return (
        len(data["staff"]),
        data["horizon"],
        len(data["qualifications"]),
        len(data["jobs"]),
    )

def get_qualification_index(list_qualifications, qualification): # qualification is "A", "B", "C" ...
    return list_qualifications.index(qualification)

In [33]:
def init_model():
    m = Model("Project modelling")
    return m

In [34]:
def create_decision_variables(model, n_staff, horizon, n_qualifs, n_jobs):
    X = model.addVars(n_staff, horizon, n_qualifs, n_jobs, vtype=GRB.BINARY, name="assignements")
    # profit 
    profit = model.addVar(lb =0, vtype = GRB.INTEGER, name = "profit")
    J = model.addVars(n_jobs, vtype=GRB.BINARY, name="completion")
    E_D = model.addVars(n_jobs, lb=0, ub=horizon, vtype=GRB.INTEGER, name="end_dates")
    L = model.addVars(n_jobs, lb=0, ub=horizon + 1, vtype=GRB.INTEGER, name="n_days_late")

    # for max_days
    S_D = model.addVars(n_jobs, lb=0, ub=horizon, vtype=GRB.INTEGER, name="start_dates")
    spans = model.addVars(n_jobs, lb=0, ub=horizon, vtype=GRB.INTEGER, name="spans")
    max_days = model.addVar(lb=0, ub=horizon, vtype=GRB.INTEGER, name="max_jobs")

    # for max_jobs 
    n_jobs_per_person = model.addVars(n_staff, lb=0, ub=n_jobs, vtype=GRB.INTEGER, name="n_jobs_per_person")
    jobs_worked_on_by_person = model.addVars(n_staff, n_jobs, vtype=GRB.BINARY, name="jobs_worked_on_by_person")
    n_worked_days_per_job_and_person = model.addVars(n_staff, n_jobs,lb=0, ub=horizon, vtype=GRB.INTEGER, name="n_worked_days_per_job_and_person")
    max_jobs = model.addVar(lb=0, ub=n_jobs, vtype=GRB.INTEGER, name="max_jobs")
    
    return model, X, J, S_D, E_D, L, n_jobs_per_person, jobs_worked_on_by_person, n_worked_days_per_job_and_person, max_jobs, max_days, spans, profit


In [35]:
def add_constraints_start_dates(model, X, S_D, n_staff, horizon, n_qualifs, n_jobs):
    model.addConstrs(
        S_D[l] <= X[i, j, k, l] * j + (1 - X[i, j, k, l]) * horizon
        for i in range(n_staff)
        for j in range(horizon) 
        for k in range(n_qualifs)
        for l in range(n_jobs)
    )

In [36]:
def add_constraints_end_dates(model, X, E_D, n_staff, horizon, n_qualifs, n_jobs):
    model.addConstrs(
        X[i, j, k, l] * j <= E_D[l] 
        for i in range(n_staff)
        for j in range(horizon)
        for k in range(n_qualifs)
        for l in range(n_jobs)
    )

In [37]:
def add_constraints_lateness(model, E_D, L, jobs, n_staff, horizon, n_qualifs, n_jobs):
    model.addConstrs(
        E_D[l] +1 - jobs[l]["due_date"] <= L[l] for l in range(n_jobs)
    )

In [38]:
def add_constraints_worked_days_below_required_days(model, X, jobs, qualifications, n_staff, horizon, n_jobs):
    model.addConstrs(quicksum(X[i,j,get_qualification_index(qualifications, k),l] for i in range(n_staff) for j in range(horizon)) <= jobs[l]["working_days_per_qualification"][k] 
                     for l in range(n_jobs) 
                     for k in jobs[l]["working_days_per_qualification"].keys())
    

In [39]:
def add_constraints_worked_days_above_required_days(model, X, J, jobs, qualifications, n_staff, horizon, n_jobs):
    model.addConstrs(quicksum(X[i,j,get_qualification_index(qualifications, k),l] for i in range(n_staff) for j in range(horizon)) >= J[l]* jobs[l]["working_days_per_qualification"][k] 
                     for l in range(n_jobs) 
                     for k in jobs[l]["working_days_per_qualification"].keys())


In [40]:
def add_constraints_employees_working_only_one_day(model, X, J, data, n_staff, n_jobs, horizon, n_qualifs):
    model.addConstrs(quicksum(X[i,j,k,l] for l in range(n_jobs) for k in range (n_qualifs)) <= 1 
                     for i in range(n_staff) 
                     for j in range(horizon))

In [41]:
def in_qualification(data, i, k):
    return data["qualifications"][k] in data["staff"][i]["qualifications"]


def add_qualification_constraints(model, n_staff, horizon, n_qualifs, n_jobs, X, data):
    model.addConstrs(
        X[i, j, k, l] == 0
        for i in range(n_staff)
        for j in range(horizon)
        for k in range(n_qualifs)
        for l in range(n_jobs)
        if not in_qualification(data, i, k)
    )


def in_vacation(i, j, data):
    data = {l: data["staff"][l] for l in range(len(data["staff"]))}
    data = data[i]["vacations"]
    return j in data


def add_vacation_constraints(model, n_staff, horizon, n_qualifs, n_jobs, X, data):
    model.addConstrs(
        X[i, j, k, l] == 0
        for i in range(n_staff)
        for j in range(horizon)
        for k in range(n_qualifs)
        for l in range(n_jobs)
        if in_vacation(i, j, data)
    )

In [42]:
def add_constraint_profit(model, J, L, profit, jobs):
    model.addConstr(profit == quicksum( (J[index_job] * job["gain"] - job["daily_penalty"] * L[index_job]) for index_job, job in enumerate(jobs)))

In [43]:
def add_constraints_n_worked_days_per_jobs_person(model, X, n_worked_days_per_job_and_person, n_staff, n_jobs, horizon, n_qualifs):
    model.addConstrs( n_worked_days_per_job_and_person[i, l] == quicksum( X[i,j,k,l] for j in range(horizon) for k in range(n_qualifs)) 
        for i in range(n_staff) 
        for l in range(n_jobs)
    )

def add_constraints_jobs_worked_on_by_person(model, jobs_worked_on_by_person, n_worked_days_per_job_and_person, n_staff, n_jobs):
    model.addConstrs((jobs_worked_on_by_person[i, l] == 0) >> (n_worked_days_per_job_and_person[i,l] == 0) 
        for i in range(n_staff) 
        for l in range(n_jobs)
    ) 
    model.addConstrs((jobs_worked_on_by_person[i, l] == 1) >> (n_worked_days_per_job_and_person[i,l] >= 1) 
        for i in range(n_staff) 
        for l in range(n_jobs)
    )

def add_constraints_n_jobs_per_person(model, n_jobs_per_person, jobs_worked_on_by_person, n_staff, n_jobs):
    model.addConstrs(n_jobs_per_person[i] == quicksum( jobs_worked_on_by_person[i, l] for l in range(n_jobs) ) for i in range(n_staff))

def add_constraint_max_jobs(model, max_jobs, n_jobs_per_person, jobs_worked_on_by_person, n_staff):
    model.addConstr(max_jobs == max_([n_jobs_per_person[i] for i in range(n_staff)]))

In [44]:
def add_constraint_spans(model, spans, E_D, S_D, n_jobs):
    model.addConstrs((spans[l] == (E_D[l] - S_D[l])+1) for l in range(n_jobs))

def add_constraint_max_days(model, max_days, spans, n_jobs):
    for l in range(n_jobs):
        model.addConstr(max_days >= spans[l]) 
    # model.addConstr(max_days == max_([spans[l] for l in range(n_jobs)]))

In [45]:
def add_profit_as_first_objective(model, profit):
    model.setObjective(
        profit,
        GRB.MAXIMIZE
    )
    return model, profit


In [46]:
def add_minimax_jobs_as_second_objective(model, max_jobs):
    model.setObjective(
        max_jobs,
        GRB.MINIMIZE
    )
    return model

In [47]:
def add_minimax_days_spent_as_third_objective(model, max_days):
    model.setObjective(
        max_days,
        GRB.MINIMIZE
    )
    return model

In [48]:
def add_mono_objective(model, profit, max_days, max_jobs):
    model.setObjective(
        10 * profit - max_days - max_jobs,
        GRB.MAXIMIZE
    )
    return model, profit, max_days, max_jobs

# Surface

In [49]:
def prepare_model(data):
    n_staff, horizon, n_qualifs, n_jobs = get_dims(data)

    # Instanciation du modèle
    model = init_model()
    
    # Création des variables : binaires dans X et J, entières de 0 à horizon + 3
    model, X, J, S_D, E_D, L, n_jobs_per_person, jobs_worked_on_by_person, n_worked_days_per_job_and_person, max_jobs, max_days, spans, profit = create_decision_variables(model, n_staff, horizon, n_qualifs, n_jobs)
    
    # maj du modèle
    model.update()
    
    # Ajout des constraintes
    
    add_constraints_employees_working_only_one_day(model, X ,J,data,n_staff,n_jobs,horizon, n_qualifs)
    add_qualification_constraints(model, n_staff, horizon, n_qualifs, n_jobs, X, data)
    add_vacation_constraints(model, n_staff, horizon, n_qualifs, n_jobs, X, data)
    
    add_constraints_start_dates(model, X, S_D, n_staff, horizon, n_qualifs, n_jobs)
    add_constraints_end_dates(model, X, E_D, n_staff, horizon, n_qualifs, n_jobs)
    add_constraints_lateness(model, E_D, L, data["jobs"], n_staff, horizon, n_qualifs, n_jobs)
    
    add_constraints_worked_days_below_required_days(model, X, data["jobs"], data["qualifications"], n_staff, horizon, n_jobs)
    add_constraints_worked_days_above_required_days(model, X , J, data["jobs"], data["qualifications"], n_staff, horizon, n_jobs)
    
    add_constraints_n_worked_days_per_jobs_person(model, X, n_worked_days_per_job_and_person, n_staff, n_jobs, horizon, n_qualifs)
    add_constraints_jobs_worked_on_by_person(model, jobs_worked_on_by_person, n_worked_days_per_job_and_person, n_staff, n_jobs)
    add_constraints_n_jobs_per_person(model, n_jobs_per_person, jobs_worked_on_by_person, n_staff, n_jobs)
    add_constraint_spans(model, spans, E_D, S_D, n_jobs)
    
    add_constraint_profit(model, J, L, profit, data["jobs"])
    add_constraint_max_jobs(model, max_jobs, n_jobs_per_person, jobs_worked_on_by_person, n_staff)
    add_constraint_max_days(model, max_days, spans, n_jobs)
    
    
    # Fonctions Objectifs
    
    # model = add_minimax_jobs_as_second_objective(model, max_jobs)
    # model = add_minimax_days_spent_as_third_objective(model, max_days)
    model, profit = add_profit_as_first_objective(model, profit)
    
    # model, profit, max_days, max_jobs = add_mono_objective(model, profit, max_days, max_jobs)
    
    # maj du modèle
    model.update()
    
    # Paramétrage (mode mute)
    # model.params.outputflag = 0

    return model, profit, max_days, max_jobs

In [50]:
def run_one_step(data, n_jobs, points, step):
    for j in tqdm(range(n_jobs, -1, -1)):
        # print(j, " / ", n_jobs)
        model, profit, max_days, max_jobs = prepare_model(data)
        model.addConstr(max_jobs <= j)
        model.addConstr(max_days <= step)
        model.update()
        model.optimize()
        point = np.array([round(step), round(j), round(profit.X)])
        points.append(point)
    return points

In [51]:
def main(data):
    n_staff, horizon, n_qualifs, n_jobs = get_dims(data)
    points = []
    for i in tqdm(range(horizon, -1,-1)):
        for j in range(n_jobs, -1, -1):
            print(j, " / ", n_jobs)
            model, profit, max_days, max_jobs = prepare_model(data)
            model.addConstr(max_jobs <= j)
            model.addConstr(max_days <= i)
            model.update()
            model.optimize()
            point = np.array([round(i), round(j), round(profit.X)])
            points.append(point)
    return points

# Computing points

In [52]:
dataset = DATASET.MEDIUM
data = load_data(dataset)
n_staff, horizon, n_qualifs, n_jobs = get_dims(data)
points = []
# points = run_one_step(data, n_jobs, points, 22)
print(points)    
    

[]


In [23]:
points = run_one_step(data, n_jobs, points, 21)
print(points)

100%|██████████| 16/16 [04:51<00:00, 18.19s/it]

[array([ 22,  15, 413]), array([ 22,  14, 413]), array([ 22,  13, 413]), array([ 22,  12, 413]), array([ 22,  11, 413]), array([ 22,  10, 413]), array([ 22,   9, 413]), array([ 22,   8, 413]), array([ 22,   7, 413]), array([ 22,   6, 413]), array([ 22,   5, 413]), array([ 22,   4, 405]), array([ 22,   3, 384]), array([ 22,   2, 326]), array([ 22,   1, 194]), array([22,  0,  0]), array([ 21,  15, 413]), array([ 21,  14, 413]), array([ 21,  13, 413]), array([ 21,  12, 413]), array([ 21,  11, 413]), array([ 21,  10, 413]), array([ 21,   9, 413]), array([ 21,   8, 413]), array([ 21,   7, 413]), array([ 21,   6, 413]), array([ 21,   5, 413]), array([ 21,   4, 405]), array([ 21,   3, 384]), array([ 21,   2, 326]), array([ 21,   1, 194]), array([21,  0,  0])]


In [24]:
points = run_one_step(data, n_jobs, points, 20)
print(points)

100%|██████████| 16/16 [03:57<00:00, 14.85s/it]

[array([ 22,  15, 413]), array([ 22,  14, 413]), array([ 22,  13, 413]), array([ 22,  12, 413]), array([ 22,  11, 413]), array([ 22,  10, 413]), array([ 22,   9, 413]), array([ 22,   8, 413]), array([ 22,   7, 413]), array([ 22,   6, 413]), array([ 22,   5, 413]), array([ 22,   4, 405]), array([ 22,   3, 384]), array([ 22,   2, 326]), array([ 22,   1, 194]), array([22,  0,  0]), array([ 21,  15, 413]), array([ 21,  14, 413]), array([ 21,  13, 413]), array([ 21,  12, 413]), array([ 21,  11, 413]), array([ 21,  10, 413]), array([ 21,   9, 413]), array([ 21,   8, 413]), array([ 21,   7, 413]), array([ 21,   6, 413]), array([ 21,   5, 413]), array([ 21,   4, 405]), array([ 21,   3, 384]), array([ 21,   2, 326]), array([ 21,   1, 194]), array([21,  0,  0]), array([ 20,  15, 413]), array([ 20,  14, 413]), array([ 20,  13, 413]), array([ 20,  12, 413]), array([ 20,  11, 413]), array([ 20,  10, 413]), array([ 20,   9, 413]), array([ 20,   8, 413]), array([ 20,   7, 413]), array([ 20,   6, 413]

In [25]:
points = run_one_step(data, n_jobs, points, 19)
print(points)

100%|██████████| 16/16 [04:15<00:00, 15.98s/it]

[array([ 22,  15, 413]), array([ 22,  14, 413]), array([ 22,  13, 413]), array([ 22,  12, 413]), array([ 22,  11, 413]), array([ 22,  10, 413]), array([ 22,   9, 413]), array([ 22,   8, 413]), array([ 22,   7, 413]), array([ 22,   6, 413]), array([ 22,   5, 413]), array([ 22,   4, 405]), array([ 22,   3, 384]), array([ 22,   2, 326]), array([ 22,   1, 194]), array([22,  0,  0]), array([ 21,  15, 413]), array([ 21,  14, 413]), array([ 21,  13, 413]), array([ 21,  12, 413]), array([ 21,  11, 413]), array([ 21,  10, 413]), array([ 21,   9, 413]), array([ 21,   8, 413]), array([ 21,   7, 413]), array([ 21,   6, 413]), array([ 21,   5, 413]), array([ 21,   4, 405]), array([ 21,   3, 384]), array([ 21,   2, 326]), array([ 21,   1, 194]), array([21,  0,  0]), array([ 20,  15, 413]), array([ 20,  14, 413]), array([ 20,  13, 413]), array([ 20,  12, 413]), array([ 20,  11, 413]), array([ 20,  10, 413]), array([ 20,   9, 413]), array([ 20,   8, 413]), array([ 20,   7, 413]), array([ 20,   6, 413]

In [26]:
points = run_one_step(data, n_jobs, points, 18)
print(points)

100%|██████████| 16/16 [04:33<00:00, 17.12s/it]

[array([ 22,  15, 413]), array([ 22,  14, 413]), array([ 22,  13, 413]), array([ 22,  12, 413]), array([ 22,  11, 413]), array([ 22,  10, 413]), array([ 22,   9, 413]), array([ 22,   8, 413]), array([ 22,   7, 413]), array([ 22,   6, 413]), array([ 22,   5, 413]), array([ 22,   4, 405]), array([ 22,   3, 384]), array([ 22,   2, 326]), array([ 22,   1, 194]), array([22,  0,  0]), array([ 21,  15, 413]), array([ 21,  14, 413]), array([ 21,  13, 413]), array([ 21,  12, 413]), array([ 21,  11, 413]), array([ 21,  10, 413]), array([ 21,   9, 413]), array([ 21,   8, 413]), array([ 21,   7, 413]), array([ 21,   6, 413]), array([ 21,   5, 413]), array([ 21,   4, 405]), array([ 21,   3, 384]), array([ 21,   2, 326]), array([ 21,   1, 194]), array([21,  0,  0]), array([ 20,  15, 413]), array([ 20,  14, 413]), array([ 20,  13, 413]), array([ 20,  12, 413]), array([ 20,  11, 413]), array([ 20,  10, 413]), array([ 20,   9, 413]), array([ 20,   8, 413]), array([ 20,   7, 413]), array([ 20,   6, 413]

In [27]:
points = run_one_step(data, n_jobs, points, 17)
print(points)

100%|██████████| 16/16 [04:10<00:00, 15.65s/it]

[array([ 22,  15, 413]), array([ 22,  14, 413]), array([ 22,  13, 413]), array([ 22,  12, 413]), array([ 22,  11, 413]), array([ 22,  10, 413]), array([ 22,   9, 413]), array([ 22,   8, 413]), array([ 22,   7, 413]), array([ 22,   6, 413]), array([ 22,   5, 413]), array([ 22,   4, 405]), array([ 22,   3, 384]), array([ 22,   2, 326]), array([ 22,   1, 194]), array([22,  0,  0]), array([ 21,  15, 413]), array([ 21,  14, 413]), array([ 21,  13, 413]), array([ 21,  12, 413]), array([ 21,  11, 413]), array([ 21,  10, 413]), array([ 21,   9, 413]), array([ 21,   8, 413]), array([ 21,   7, 413]), array([ 21,   6, 413]), array([ 21,   5, 413]), array([ 21,   4, 405]), array([ 21,   3, 384]), array([ 21,   2, 326]), array([ 21,   1, 194]), array([21,  0,  0]), array([ 20,  15, 413]), array([ 20,  14, 413]), array([ 20,  13, 413]), array([ 20,  12, 413]), array([ 20,  11, 413]), array([ 20,  10, 413]), array([ 20,   9, 413]), array([ 20,   8, 413]), array([ 20,   7, 413]), array([ 20,   6, 413]

In [28]:
points = run_one_step(data, n_jobs, points, 16)
print(points)

100%|██████████| 16/16 [03:45<00:00, 14.10s/it]

[array([ 22,  15, 413]), array([ 22,  14, 413]), array([ 22,  13, 413]), array([ 22,  12, 413]), array([ 22,  11, 413]), array([ 22,  10, 413]), array([ 22,   9, 413]), array([ 22,   8, 413]), array([ 22,   7, 413]), array([ 22,   6, 413]), array([ 22,   5, 413]), array([ 22,   4, 405]), array([ 22,   3, 384]), array([ 22,   2, 326]), array([ 22,   1, 194]), array([22,  0,  0]), array([ 21,  15, 413]), array([ 21,  14, 413]), array([ 21,  13, 413]), array([ 21,  12, 413]), array([ 21,  11, 413]), array([ 21,  10, 413]), array([ 21,   9, 413]), array([ 21,   8, 413]), array([ 21,   7, 413]), array([ 21,   6, 413]), array([ 21,   5, 413]), array([ 21,   4, 405]), array([ 21,   3, 384]), array([ 21,   2, 326]), array([ 21,   1, 194]), array([21,  0,  0]), array([ 20,  15, 413]), array([ 20,  14, 413]), array([ 20,  13, 413]), array([ 20,  12, 413]), array([ 20,  11, 413]), array([ 20,  10, 413]), array([ 20,   9, 413]), array([ 20,   8, 413]), array([ 20,   7, 413]), array([ 20,   6, 413]

In [29]:
points = run_one_step(data, n_jobs, points, 15)
print(points)

100%|██████████| 16/16 [04:05<00:00, 15.32s/it]

[array([ 22,  15, 413]), array([ 22,  14, 413]), array([ 22,  13, 413]), array([ 22,  12, 413]), array([ 22,  11, 413]), array([ 22,  10, 413]), array([ 22,   9, 413]), array([ 22,   8, 413]), array([ 22,   7, 413]), array([ 22,   6, 413]), array([ 22,   5, 413]), array([ 22,   4, 405]), array([ 22,   3, 384]), array([ 22,   2, 326]), array([ 22,   1, 194]), array([22,  0,  0]), array([ 21,  15, 413]), array([ 21,  14, 413]), array([ 21,  13, 413]), array([ 21,  12, 413]), array([ 21,  11, 413]), array([ 21,  10, 413]), array([ 21,   9, 413]), array([ 21,   8, 413]), array([ 21,   7, 413]), array([ 21,   6, 413]), array([ 21,   5, 413]), array([ 21,   4, 405]), array([ 21,   3, 384]), array([ 21,   2, 326]), array([ 21,   1, 194]), array([21,  0,  0]), array([ 20,  15, 413]), array([ 20,  14, 413]), array([ 20,  13, 413]), array([ 20,  12, 413]), array([ 20,  11, 413]), array([ 20,  10, 413]), array([ 20,   9, 413]), array([ 20,   8, 413]), array([ 20,   7, 413]), array([ 20,   6, 413]

In [30]:
points = run_one_step(data, n_jobs, points, 14)
print(points)

100%|██████████| 16/16 [05:01<00:00, 18.85s/it]

[array([ 22,  15, 413]), array([ 22,  14, 413]), array([ 22,  13, 413]), array([ 22,  12, 413]), array([ 22,  11, 413]), array([ 22,  10, 413]), array([ 22,   9, 413]), array([ 22,   8, 413]), array([ 22,   7, 413]), array([ 22,   6, 413]), array([ 22,   5, 413]), array([ 22,   4, 405]), array([ 22,   3, 384]), array([ 22,   2, 326]), array([ 22,   1, 194]), array([22,  0,  0]), array([ 21,  15, 413]), array([ 21,  14, 413]), array([ 21,  13, 413]), array([ 21,  12, 413]), array([ 21,  11, 413]), array([ 21,  10, 413]), array([ 21,   9, 413]), array([ 21,   8, 413]), array([ 21,   7, 413]), array([ 21,   6, 413]), array([ 21,   5, 413]), array([ 21,   4, 405]), array([ 21,   3, 384]), array([ 21,   2, 326]), array([ 21,   1, 194]), array([21,  0,  0]), array([ 20,  15, 413]), array([ 20,  14, 413]), array([ 20,  13, 413]), array([ 20,  12, 413]), array([ 20,  11, 413]), array([ 20,  10, 413]), array([ 20,   9, 413]), array([ 20,   8, 413]), array([ 20,   7, 413]), array([ 20,   6, 413]

In [31]:
points = run_one_step(data, n_jobs, points, 13)
print(points)

100%|██████████| 16/16 [03:32<00:00, 13.26s/it]

[array([ 22,  15, 413]), array([ 22,  14, 413]), array([ 22,  13, 413]), array([ 22,  12, 413]), array([ 22,  11, 413]), array([ 22,  10, 413]), array([ 22,   9, 413]), array([ 22,   8, 413]), array([ 22,   7, 413]), array([ 22,   6, 413]), array([ 22,   5, 413]), array([ 22,   4, 405]), array([ 22,   3, 384]), array([ 22,   2, 326]), array([ 22,   1, 194]), array([22,  0,  0]), array([ 21,  15, 413]), array([ 21,  14, 413]), array([ 21,  13, 413]), array([ 21,  12, 413]), array([ 21,  11, 413]), array([ 21,  10, 413]), array([ 21,   9, 413]), array([ 21,   8, 413]), array([ 21,   7, 413]), array([ 21,   6, 413]), array([ 21,   5, 413]), array([ 21,   4, 405]), array([ 21,   3, 384]), array([ 21,   2, 326]), array([ 21,   1, 194]), array([21,  0,  0]), array([ 20,  15, 413]), array([ 20,  14, 413]), array([ 20,  13, 413]), array([ 20,  12, 413]), array([ 20,  11, 413]), array([ 20,  10, 413]), array([ 20,   9, 413]), array([ 20,   8, 413]), array([ 20,   7, 413]), array([ 20,   6, 413]

In [32]:
points = run_one_step(data, n_jobs, points, 12)
print(points)

100%|██████████| 16/16 [05:09<00:00, 19.37s/it]

[array([ 22,  15, 413]), array([ 22,  14, 413]), array([ 22,  13, 413]), array([ 22,  12, 413]), array([ 22,  11, 413]), array([ 22,  10, 413]), array([ 22,   9, 413]), array([ 22,   8, 413]), array([ 22,   7, 413]), array([ 22,   6, 413]), array([ 22,   5, 413]), array([ 22,   4, 405]), array([ 22,   3, 384]), array([ 22,   2, 326]), array([ 22,   1, 194]), array([22,  0,  0]), array([ 21,  15, 413]), array([ 21,  14, 413]), array([ 21,  13, 413]), array([ 21,  12, 413]), array([ 21,  11, 413]), array([ 21,  10, 413]), array([ 21,   9, 413]), array([ 21,   8, 413]), array([ 21,   7, 413]), array([ 21,   6, 413]), array([ 21,   5, 413]), array([ 21,   4, 405]), array([ 21,   3, 384]), array([ 21,   2, 326]), array([ 21,   1, 194]), array([21,  0,  0]), array([ 20,  15, 413]), array([ 20,  14, 413]), array([ 20,  13, 413]), array([ 20,  12, 413]), array([ 20,  11, 413]), array([ 20,  10, 413]), array([ 20,   9, 413]), array([ 20,   8, 413]), array([ 20,   7, 413]), array([ 20,   6, 413]

In [33]:
points = run_one_step(data, n_jobs, points, 11)
print(points)

100%|██████████| 16/16 [03:57<00:00, 14.81s/it]

[array([ 22,  15, 413]), array([ 22,  14, 413]), array([ 22,  13, 413]), array([ 22,  12, 413]), array([ 22,  11, 413]), array([ 22,  10, 413]), array([ 22,   9, 413]), array([ 22,   8, 413]), array([ 22,   7, 413]), array([ 22,   6, 413]), array([ 22,   5, 413]), array([ 22,   4, 405]), array([ 22,   3, 384]), array([ 22,   2, 326]), array([ 22,   1, 194]), array([22,  0,  0]), array([ 21,  15, 413]), array([ 21,  14, 413]), array([ 21,  13, 413]), array([ 21,  12, 413]), array([ 21,  11, 413]), array([ 21,  10, 413]), array([ 21,   9, 413]), array([ 21,   8, 413]), array([ 21,   7, 413]), array([ 21,   6, 413]), array([ 21,   5, 413]), array([ 21,   4, 405]), array([ 21,   3, 384]), array([ 21,   2, 326]), array([ 21,   1, 194]), array([21,  0,  0]), array([ 20,  15, 413]), array([ 20,  14, 413]), array([ 20,  13, 413]), array([ 20,  12, 413]), array([ 20,  11, 413]), array([ 20,  10, 413]), array([ 20,   9, 413]), array([ 20,   8, 413]), array([ 20,   7, 413]), array([ 20,   6, 413]

In [34]:
points = run_one_step(data, n_jobs, points, 10)
print(points)

100%|██████████| 16/16 [07:00<00:00, 26.31s/it]

[array([ 22,  15, 413]), array([ 22,  14, 413]), array([ 22,  13, 413]), array([ 22,  12, 413]), array([ 22,  11, 413]), array([ 22,  10, 413]), array([ 22,   9, 413]), array([ 22,   8, 413]), array([ 22,   7, 413]), array([ 22,   6, 413]), array([ 22,   5, 413]), array([ 22,   4, 405]), array([ 22,   3, 384]), array([ 22,   2, 326]), array([ 22,   1, 194]), array([22,  0,  0]), array([ 21,  15, 413]), array([ 21,  14, 413]), array([ 21,  13, 413]), array([ 21,  12, 413]), array([ 21,  11, 413]), array([ 21,  10, 413]), array([ 21,   9, 413]), array([ 21,   8, 413]), array([ 21,   7, 413]), array([ 21,   6, 413]), array([ 21,   5, 413]), array([ 21,   4, 405]), array([ 21,   3, 384]), array([ 21,   2, 326]), array([ 21,   1, 194]), array([21,  0,  0]), array([ 20,  15, 413]), array([ 20,  14, 413]), array([ 20,  13, 413]), array([ 20,  12, 413]), array([ 20,  11, 413]), array([ 20,  10, 413]), array([ 20,   9, 413]), array([ 20,   8, 413]), array([ 20,   7, 413]), array([ 20,   6, 413]

In [35]:
points = run_one_step(data, n_jobs, points, 9)
print(points)

100%|██████████| 16/16 [05:31<00:00, 20.69s/it]

[array([ 22,  15, 413]), array([ 22,  14, 413]), array([ 22,  13, 413]), array([ 22,  12, 413]), array([ 22,  11, 413]), array([ 22,  10, 413]), array([ 22,   9, 413]), array([ 22,   8, 413]), array([ 22,   7, 413]), array([ 22,   6, 413]), array([ 22,   5, 413]), array([ 22,   4, 405]), array([ 22,   3, 384]), array([ 22,   2, 326]), array([ 22,   1, 194]), array([22,  0,  0]), array([ 21,  15, 413]), array([ 21,  14, 413]), array([ 21,  13, 413]), array([ 21,  12, 413]), array([ 21,  11, 413]), array([ 21,  10, 413]), array([ 21,   9, 413]), array([ 21,   8, 413]), array([ 21,   7, 413]), array([ 21,   6, 413]), array([ 21,   5, 413]), array([ 21,   4, 405]), array([ 21,   3, 384]), array([ 21,   2, 326]), array([ 21,   1, 194]), array([21,  0,  0]), array([ 20,  15, 413]), array([ 20,  14, 413]), array([ 20,  13, 413]), array([ 20,  12, 413]), array([ 20,  11, 413]), array([ 20,  10, 413]), array([ 20,   9, 413]), array([ 20,   8, 413]), array([ 20,   7, 413]), array([ 20,   6, 413]

In [42]:
points = run_one_step(data, n_jobs, points, 8)
print(points)

  0%|          | 0/16 [00:00<?, ?it/s]

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: AMD Ryzen 5 5600H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 45066 rows, 16733 columns and 117817 nonzeros
Model fingerprint: 0xd2f0dd91
Model has 151 general constraints
Variable types: 0 continuous, 16733 integer (16590 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 2e+01]
  GenCon rhs range [1e+00, 1e+00]
  GenCon coe range [1e+00, 1e+00]
Presolve removed 41627 rows and 13697 columns
Presolve time: 0.16s
Presolved: 3439 rows, 3036 columns, 17309 nonzeros
Variable types: 0 continuous, 3036 integer (2898 binary)
Found heuristic solution: objective -1.0000000

Root relaxation: objective 4.446214e+03, 2971 iterations, 0.11 seconds (0.22 work units)

    Nodes    |    Current Node    |     Objectiv

KeyboardInterrupt: 

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "C:\Users\guilh\AppData\Roaming\Python\Python39\site-packages\ipykernel\iostream.py", line 518, in write
    def write(self, string: str) -> Optional[int]:  # type:ignore[override]
KeyboardInterrupt: 


     0     0 4370.38066    0  310 3437.00000 4370.38066  27.2%     -    1s
H    0     0                    3656.0000000 4369.86739  19.5%     -    1s
     0     0 4369.86739    0  278 3656.00000 4369.86739  19.5%     -    1s
     0     0 4369.61321    0  276 3656.00000 4369.61321  19.5%     -    1s
     0     0 4369.55121    0  274 3656.00000 4369.55121  19.5%     -    1s
     0     0 4369.55121    0  275 3656.00000 4369.55121  19.5%     -    1s
     0     0 4351.21923    0  311 3656.00000 4351.21923  19.0%     -    1s
     0     0 4350.93718    0  308 3656.00000 4350.93718  19.0%     -    1s
     0     0 4349.90571    0  295 3656.00000 4349.90571  19.0%     -    1s
     0     0 4349.06748    0  284 3656.00000 4349.06748  19.0%     -    1s
     0     0 4349.06748    0  298 3656.00000 4349.06748  19.0%     -    1s
H    0     0                    3687.0000000 4328.81329  17.4%     -    2s
     0     0 4328.81329    0  332 3687.00000 4328.81329  17.4%     -    2s
H    0     0             

...

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: AMD Ryzen 5 5600H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 45066 rows, 16733 columns and 117817 nonzeros
Model fingerprint: 0xa6c969ab
Model has 151 general constraints
Variable types: 0 continuous, 16733 integer (16590 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 2e+01]
  GenCon rhs range [1e+00, 1e+00]
  GenCon coe range [1e+00, 1e+00]
Presolve removed 41627 rows and 13697 columns
Presolve time: 0.19s
Presolved: 3439 rows, 3036 columns, 17309 nonzeros
Variable types: 0 continuous, 3036 integer (2899 binary)
Found heuristic solution: objective -1.0000000

Root relaxation: objective 4.446214e+03, 3078 iterations, 0.11 seconds (0.19 work units)

    Nodes    |    Current Node    |     Objectiv

 94%|█████████▍| 15/16 [12:27<01:08, 68.04s/it]

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: AMD Ryzen 5 5600H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 45066 rows, 16733 columns and 117817 nonzeros
Model fingerprint: 0xb272fdc5
Model has 151 general constraints
Variable types: 0 continuous, 16733 integer (16590 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 2e+01]
  GenCon rhs range [1e+00, 1e+00]
  GenCon coe range [1e+00, 1e+00]
Presolve removed 45066 rows and 16733 columns
Presolve time: 0.06s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.07 seconds (0.09 work units)
Thread count was 1 (of 12 available processors)

Solution count 1: -0 
No other solutions better than -0

Optimal solution found (tolerance 1.00e-04)
Best objective -0.000000000000

100%|██████████| 16/16 [12:28<00:00, 46.76s/it]

[array([ 22,  15, 413]), array([ 22,  14, 413]), array([ 22,  13, 413]), array([ 22,  12, 413]), array([ 22,  11, 413]), array([ 22,  10, 413]), array([ 22,   9, 413]), array([ 22,   8, 413]), array([ 22,   7, 413]), array([ 22,   6, 413]), array([ 22,   5, 413]), array([ 22,   4, 405]), array([ 22,   3, 384]), array([ 22,   2, 326]), array([ 22,   1, 194]), array([22,  0,  0]), array([ 21,  15, 413]), array([ 21,  14, 413]), array([ 21,  13, 413]), array([ 21,  12, 413]), array([ 21,  11, 413]), array([ 21,  10, 413]), array([ 21,   9, 413]), array([ 21,   8, 413]), array([ 21,   7, 413]), array([ 21,   6, 413]), array([ 21,   5, 413]), array([ 21,   4, 405]), array([ 21,   3, 384]), array([ 21,   2, 326]), array([ 21,   1, 194]), array([21,  0,  0]), array([ 20,  15, 413]), array([ 20,  14, 413]), array([ 20,  13, 413]), array([ 20,  12, 413]), array([ 20,  11, 413]), array([ 20,  10, 413]), array([ 20,   9, 413]), array([ 20,   8, 413]), array([ 20,   7, 413]), array([ 20,   6, 413]

In [26]:
print(points)

[[ 22  15 413]
 [ 22  14 413]
 [ 22  13 413]
 [ 22  12 413]
 [ 22  11 413]
 [ 22  10 413]
 [ 22   9 413]
 [ 22   8 413]
 [ 22   7 413]
 [ 22   6 413]
 [ 22   5 413]
 [ 22   4 405]
 [ 22   3 384]
 [ 22   2 326]
 [ 22   1 194]
 [ 22   0   0]
 [ 21  15 413]
 [ 21  14 413]
 [ 21  13 413]
 [ 21  12 413]
 [ 21  11 413]
 [ 21  10 413]
 [ 21   9 413]
 [ 21   8 413]
 [ 21   7 413]
 [ 21   6 413]
 [ 21   5 413]
 [ 21   4 405]
 [ 21   3 384]
 [ 21   2 326]
 [ 21   1 194]
 [ 21   0   0]
 [ 20  15 413]
 [ 20  14 413]
 [ 20  13 413]
 [ 20  12 413]
 [ 20  11 413]
 [ 20  10 413]
 [ 20   9 413]
 [ 20   8 413]
 [ 20   7 413]
 [ 20   6 413]
 [ 20   5 413]
 [ 20   4 405]
 [ 20   3 384]
 [ 20   2 326]
 [ 20   1 194]
 [ 20   0   0]
 [ 19  15 413]
 [ 19  14 413]
 [ 19  13 413]
 [ 19  12 413]
 [ 19  11 413]
 [ 19  10 413]
 [ 19   9 413]
 [ 19   8 413]
 [ 19   7 413]
 [ 19   6 413]
 [ 19   5 413]
 [ 19   4 405]
 [ 19   3 384]
 [ 19   2 326]
 [ 19   1 194]
 [ 19   0   0]
 [ 18  15 413]
 [ 18  14 413]
 [ 18  13 

In [54]:
points = run_one_step(data, n_jobs, points, 7)
print(points)

  0%|          | 0/16 [00:00<?, ?it/s]

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: AMD Ryzen 5 5600H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 45066 rows, 16733 columns and 117817 nonzeros
Model fingerprint: 0xff085b00
Model has 151 general constraints
Variable types: 0 continuous, 16733 integer (16590 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 2e+01]
  GenCon rhs range [1e+00, 1e+00]
  GenCon coe range [1e+00, 1e+00]
Presolve removed 43255 rows and 15359 columns
Presolve time: 0.18s
Presolved: 1811 rows, 1374 columns, 8263 nonzeros
Variable types: 0 continuous, 1374 integer (1313 binary)
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 27.0000000

Root relaxation: objective 4.448214e+02, 949 iterations, 0.02 seconds (0.02 work units)

   

  6%|▋         | 1/16 [00:09<02:27,  9.84s/it]

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: AMD Ryzen 5 5600H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 45066 rows, 16733 columns and 117817 nonzeros
Model fingerprint: 0xd1e9abf4
Model has 151 general constraints
Variable types: 0 continuous, 16733 integer (16590 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 2e+01]
  GenCon rhs range [1e+00, 1e+00]
  GenCon coe range [1e+00, 1e+00]
Presolve removed 41642 rows and 13699 columns
Presolve time: 0.16s
Presolved: 3424 rows, 3034 columns, 17274 nonzeros
Variable types: 0 continuous, 3034 integer (2898 binary)
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 27.0000000

Root relaxation: objective 4.448214e+02, 3036 iterations, 0.10 seconds (0.16 work units)

 

 12%|█▎        | 2/16 [00:19<02:17,  9.84s/it]

...

 69%|██████▉   | 11/16 [03:22<03:05, 37.12s/it]

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: AMD Ryzen 5 5600H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 45066 rows, 16733 columns and 117817 nonzeros
Model fingerprint: 0x54db9796
Model has 151 general constraints
Variable types: 0 continuous, 16733 integer (16590 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 2e+01]
  GenCon rhs range [1e+00, 1e+00]
  GenCon coe range [1e+00, 1e+00]
Presolve removed 41642 rows and 13699 columns
Presolve time: 0.16s
Presolved: 3424 rows, 3034 columns, 17274 nonzeros
Variable types: 0 continuous, 3034 integer (2898 binary)
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 27.0000000

Root relaxation: objective 4.448214e+02, 2225 iterations, 0.08 seconds (0.15 work units)

 

KeyboardInterrupt: 

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "C:\Users\guilh\AppData\Roaming\Python\Python39\site-packages\ipykernel\iostream.py", line 518, in write
    def write(self, string: str) -> Optional[int]:  # type:ignore[override]
KeyboardInterrupt: 


 355992 94947 infeasible   53       385.00000  403.00000  4.68%  70.6  266s


In [ ]:
points = run_one_step(data, n_jobs, points, 6)
print(points)

In [ ]:
points = run_one_step(data, n_jobs, points, 5)
print(points)

In [ ]:
points = run_one_step(data, n_jobs, points, 4)
print(points)

In [ ]:
points = run_one_step(data, n_jobs, points, 3)
print(points)

In [ ]:
points = run_one_step(data, n_jobs, points, 2)
print(points)

In [ ]:
points = run_one_step(data, n_jobs, points, 1)
print(points)

In [ ]:
points = run_one_step(data, n_jobs, points, 0)
print(points)

In [ ]:
points = np.array(points)

  0%|          | 0/23 [00:00<?, ?it/s]

15  /  15
Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-10


  0%|          | 0/23 [00:32<?, ?it/s]


KeyboardInterrupt: 

# Compute with main

In [23]:
# dataset = DATASET.MEDIUM
# data = load_data(dataset)
# n_staff, horizon, n_qualifs, n_jobs = get_dims(data)
# points = main(data)
# print(points)

  0%|          | 0/23 [00:00<?, ?it/s]

15  /  15
Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-10
14  /  15
13  /  15
12  /  15
11  /  15
10  /  15
9  /  15
8  /  15
7  /  15
6  /  15
5  /  15
4  /  15
3  /  15
2  /  15
1  /  15
0  /  15


  4%|▍         | 1/23 [01:32<33:54, 92.50s/it]

15  /  15
14  /  15
13  /  15
12  /  15
11  /  15
10  /  15
9  /  15
8  /  15
7  /  15
6  /  15
5  /  15
4  /  15
3  /  15
2  /  15
1  /  15
0  /  15


  9%|▊         | 2/23 [03:02<31:47, 90.81s/it]

...

15  /  15
14  /  15
13  /  15
12  /  15
11  /  15
10  /  15
9  /  15
8  /  15
7  /  15
6  /  15
5  /  15
4  /  15
3  /  15
2  /  15
1  /  15
0  /  15


 61%|██████    | 14/23 [23:10<19:57, 133.01s/it]

15  /  15
14  /  15
13  /  15
12  /  15
11  /  15
10  /  15
9  /  15
8  /  15
7  /  15
6  /  15
5  /  15
4  /  15
3  /  15
2  /  15
1  /  15
0  /  15


 65%|██████▌   | 15/23 [28:32<25:19, 189.88s/it]

15  /  15
14  /  15
13  /  15
12  /  15
11  /  15
10  /  15
9  /  15
8  /  15
7  /  15
6  /  15
5  /  15
4  /  15


# Visualisation

In [53]:
# points = [array([ 22,  15, 413]), array([ 22,  14, 413]), array([ 22,  13, 413]), array([ 22,  12, 413]), array([ 22,  11, 413]), array([ 22,  10, 413]), array([ 22,   9, 413]), array([ 22,   8, 413]), array([ 22,   7, 413]), array([ 22,   6, 413]), array([ 22,   5, 413]), array([ 22,   4, 405]), array([ 22,   3, 384]), array([ 22,   2, 326]), array([ 22,   1, 194]), array([22,  0,  0]), array([ 21,  15, 413]), array([ 21,  14, 413]), array([ 21,  13, 413]), array([ 21,  12, 413]), array([ 21,  11, 413]), array([ 21,  10, 413]), array([ 21,   9, 413]), array([ 21,   8, 413]), array([ 21,   7, 413]), array([ 21,   6, 413]), array([ 21,   5, 413]), array([ 21,   4, 405]), array([ 21,   3, 384]), array([ 21,   2, 326]), array([ 21,   1, 194]), array([21,  0,  0]), array([ 20,  15, 413]), array([ 20,  14, 413]), array([ 20,  13, 413]), array([ 20,  12, 413]), array([ 20,  11, 413]), array([ 20,  10, 413]), array([ 20,   9, 413]), array([ 20,   8, 413]), array([ 20,   7, 413]), array([ 20,   6, 413]), array([ 20,   5, 413]), array([ 20,   4, 405]), array([ 20,   3, 384]), array([ 20,   2, 326]), array([ 20,   1, 194]), array([20,  0,  0]), array([ 19,  15, 413]), array([ 19,  14, 413]), array([ 19,  13, 413]), array([ 19,  12, 413]), array([ 19,  11, 413]), array([ 19,  10, 413]), array([ 19,   9, 413]), array([ 19,   8, 413]), array([ 19,   7, 413]), array([ 19,   6, 413]), array([ 19,   5, 413]), array([ 19,   4, 405]), array([ 19,   3, 384]), array([ 19,   2, 326]), array([ 19,   1, 194]), array([19,  0,  0]), array([ 18,  15, 413]), array([ 18,  14, 413]), array([ 18,  13, 413]), array([ 18,  12, 413]), array([ 18,  11, 413]), array([ 18,  10, 413]), array([ 18,   9, 413]), array([ 18,   8, 413]), array([ 18,   7, 413]), array([ 18,   6, 413]), array([ 18,   5, 413]), array([ 18,   4, 405]), array([ 18,   3, 384]), array([ 18,   2, 326]), array([ 18,   1, 194]), array([18,  0,  0]), array([ 17,  15, 413]), array([ 17,  14, 413]), array([ 17,  13, 413]), array([ 17,  12, 413]), array([ 17,  11, 413]), array([ 17,  10, 413]), array([ 17,   9, 413]), array([ 17,   8, 413]), array([ 17,   7, 413]), array([ 17,   6, 413]), array([ 17,   5, 413]), array([ 17,   4, 405]), array([ 17,   3, 384]), array([ 17,   2, 326]), array([ 17,   1, 194]), array([17,  0,  0]), array([ 16,  15, 413]), array([ 16,  14, 413]), array([ 16,  13, 413]), array([ 16,  12, 413]), array([ 16,  11, 413]), array([ 16,  10, 413]), array([ 16,   9, 413]), array([ 16,   8, 413]), array([ 16,   7, 413]), array([ 16,   6, 413]), array([ 16,   5, 413]), array([ 16,   4, 405]), array([ 16,   3, 384]), array([ 16,   2, 326]), array([ 16,   1, 194]), array([16,  0,  0]), array([ 15,  15, 413]), array([ 15,  14, 413]), array([ 15,  13, 413]), array([ 15,  12, 413]), array([ 15,  11, 413]), array([ 15,  10, 413]), array([ 15,   9, 413]), array([ 15,   8, 413]), array([ 15,   7, 413]), array([ 15,   6, 413]), array([ 15,   5, 413]), array([ 15,   4, 405]), array([ 15,   3, 384]), array([ 15,   2, 326]), array([ 15,   1, 194]), array([15,  0,  0]), array([ 14,  15, 413]), array([ 14,  14, 413]), array([ 14,  13, 413]), array([ 14,  12, 413]), array([ 14,  11, 413]), array([ 14,  10, 413]), array([ 14,   9, 413]), array([ 14,   8, 413]), array([ 14,   7, 413]), array([ 14,   6, 413]), array([ 14,   5, 413]), array([ 14,   4, 405]), array([ 14,   3, 384]), array([ 14,   2, 326]), array([ 14,   1, 194]), array([14,  0,  0]), array([ 13,  15, 413]), array([ 13,  14, 413]), array([ 13,  13, 413]), array([ 13,  12, 413]), array([ 13,  11, 413]), array([ 13,  10, 413]), array([ 13,   9, 413]), array([ 13,   8, 413]), array([ 13,   7, 413]), array([ 13,   6, 413]), array([ 13,   5, 413]), array([ 13,   4, 405]), array([ 13,   3, 384]), array([ 13,   2, 326]), array([ 13,   1, 194]), array([13,  0,  0]), array([ 12,  15, 413]), array([ 12,  14, 413]), array([ 12,  13, 413]), array([ 12,  12, 413]), array([ 12,  11, 413]), array([ 12,  10, 413]), array([ 12,   9, 413]), array([ 12,   8, 413]), array([ 12,   7, 413]), array([ 12,   6, 413]), array([ 12,   5, 413]), array([ 12,   4, 405]), array([ 12,   3, 384]), array([ 12,   2, 326]), array([ 12,   1, 194]), array([12,  0,  0]), array([ 11,  15, 413]), array([ 11,  14, 413]), array([ 11,  13, 413]), array([ 11,  12, 413]), array([ 11,  11, 413]), array([ 11,  10, 413]), array([ 11,   9, 413]), array([ 11,   8, 413]), array([ 11,   7, 413]), array([ 11,   6, 413]), array([ 11,   5, 413]), array([ 11,   4, 404]), array([ 11,   3, 379]), array([ 11,   2, 325]), array([ 11,   1, 180]), array([11,  0,  0]), array([ 10,  15, 413]), array([ 10,  14, 413]), array([ 10,  13, 413]), array([ 10,  12, 413]), array([ 10,  11, 413]), array([ 10,  10, 413]), array([ 10,   9, 413]), array([ 10,   8, 413]), array([ 10,   7, 413]), array([ 10,   6, 413]), array([ 10,   5, 413]), array([ 10,   4, 404]), array([ 10,   3, 379]), array([ 10,   2, 325]), array([ 10,   1, 180]), array([10,  0,  0]), array([  9,  15, 413]), array([  9,  14, 413]), array([  9,  13, 413]), array([  9,  12, 413]), array([  9,  11, 413]), array([  9,  10, 413]), array([  9,   9, 413]), array([  9,   8, 413]), array([  9,   7, 413]), array([  9,   6, 413]), array([  9,   5, 410]), array([  9,   4, 400]), array([  9,   3, 379]), array([  9,   2, 325]), array([  9,   1, 180]), array([9, 0, 0]), array([  8,  15, 413]), array([  8,  14, 413]), array([  8,  13, 413]), array([  8,  12, 413]), array([  8,  11, 413]), array([  8,  10, 413]), array([  8,   9, 413]), array([  8,   8, 413]), array([  8,   7, 413]), array([  8,   6, 413]), array([  8,   5, 410]), array([  8,   4, 397]), array([  8,   3, 369]), array([  8,   2, 302]), array([  8,   1, 177]), array([8, 0, 0])]
# points = array(points)
print(points)

[array([ 22,  15, 413]), array([ 22,  14, 413]), array([ 22,  13, 413]), array([ 22,  12, 413]), array([ 22,  11, 413]), array([ 22,  10, 413]), array([ 22,   9, 413]), array([ 22,   8, 413]), array([ 22,   7, 413]), array([ 22,   6, 413]), array([ 22,   5, 413]), array([ 22,   4, 405]), array([ 22,   3, 384]), array([ 22,   2, 326]), array([ 22,   1, 194]), array([22,  0,  0]), array([ 21,  15, 413]), array([ 21,  14, 413]), array([ 21,  13, 413]), array([ 21,  12, 413]), array([ 21,  11, 413]), array([ 21,  10, 413]), array([ 21,   9, 413]), array([ 21,   8, 413]), array([ 21,   7, 413]), array([ 21,   6, 413]), array([ 21,   5, 413]), array([ 21,   4, 405]), array([ 21,   3, 384]), array([ 21,   2, 326]), array([ 21,   1, 194]), array([21,  0,  0]), array([ 20,  15, 413]), array([ 20,  14, 413]), array([ 20,  13, 413]), array([ 20,  12, 413]), array([ 20,  11, 413]), array([ 20,  10, 413]), array([ 20,   9, 413]), array([ 20,   8, 413]), array([ 20,   7, 413]), array([ 20,   6, 413]

In [14]:
Z = np.array(points[:,2]).reshape(horizon+1,n_jobs+1)

X = np.linspace(horizon,0,horizon +1)
Y = np.linspace(n_jobs,0,n_jobs +1)
X, Y = np.meshgrid(X, Y)

print(X)
print(Y)
print(Z)


fig = go.Figure(data=[go.Surface(x=X, y=Y, z=Z.T)])
fig.update_layout(scene=dict(xaxis_title="Jobs", yaxis_title="Days", zaxis_title="Benefit"))
fig.show()

[[15. 14. 13. 12. 11. 10.  9.  8.  7.  6.  5.  4.  3.  2.  1.  0.]
 [15. 14. 13. 12. 11. 10.  9.  8.  7.  6.  5.  4.  3.  2.  1.  0.]
 [15. 14. 13. 12. 11. 10.  9.  8.  7.  6.  5.  4.  3.  2.  1.  0.]
 [15. 14. 13. 12. 11. 10.  9.  8.  7.  6.  5.  4.  3.  2.  1.  0.]
 [15. 14. 13. 12. 11. 10.  9.  8.  7.  6.  5.  4.  3.  2.  1.  0.]
 [15. 14. 13. 12. 11. 10.  9.  8.  7.  6.  5.  4.  3.  2.  1.  0.]
 [15. 14. 13. 12. 11. 10.  9.  8.  7.  6.  5.  4.  3.  2.  1.  0.]
 [15. 14. 13. 12. 11. 10.  9.  8.  7.  6.  5.  4.  3.  2.  1.  0.]
 [15. 14. 13. 12. 11. 10.  9.  8.  7.  6.  5.  4.  3.  2.  1.  0.]
 [15. 14. 13. 12. 11. 10.  9.  8.  7.  6.  5.  4.  3.  2.  1.  0.]
 [15. 14. 13. 12. 11. 10.  9.  8.  7.  6.  5.  4.  3.  2.  1.  0.]
 [15. 14. 13. 12. 11. 10.  9.  8.  7.  6.  5.  4.  3.  2.  1.  0.]
 [15. 14. 13. 12. 11. 10.  9.  8.  7.  6.  5.  4.  3.  2.  1.  0.]
 [15. 14. 13. 12. 11. 10.  9.  8.  7.  6.  5.  4.  3.  2.  1.  0.]
 [15. 14. 13. 12. 11. 10.  9.  8.  7.  6.  5.  4.  3.  2.  1. 